<h1>Interaction with Claude API</h1>


<h2>Setup</h2>
Install anthropic and python-dotenv libraries<br>
python-dotenv allows program to load configuration data from .env file. <br>

In [ ]:
%pip install anthropic python-dotenv

In [31]:
from anthropic import Anthropic

class AntrophicPrompt():

    def __init__(self, model="claude-3-5-haiku-latest"):
        self.model = model
        self.client = Anthropic()
        self.messages = []
        self.user_message = None
        self.assistant_message = None
        self.system_message = None

    def add_user_message(self, text):
        self.user_message = {"role": "user", "content": text}
        self.messages.append(self.user_message)


    def add_assistant_message(self, text):
        self.assistant_message = {"role": "assistant", "content": text}
        self.messages.append(self.assistant_message)


    def set_system_message(self, text):
        self.system_message = text

    def gen_completion(self, system=None, temperature=1.0, stop_sequences=[]):
        params = {
            "model": self.model,
            "max_tokens": 1000,
            "messages": self.messages,
            "temperature": temperature,
            "stop_sequences": stop_sequences,
        }

        if system:
            params["system"] = system
        elif self.system_message:
            params["system"] = self.system_message

        message = self.client.messages.create(**params)
        return message.content[0].text

    def clear_history(self):
        self.messages = []
        self.user_message = None
        self.assistant_message = None
        self.system_message = None
        self.model = "claude-3-5-haiku-latest"

Get API Key

In [15]:
# Load API Key
from dotenv import load_dotenv
load_dotenv()

True

Let's test

In [32]:
#
model = "claude-sonnet-4-0"
client = AntrophicPrompt(model)

user_prompt = "Define quantum computing in one sentence"
client.add_user_message(user_prompt)

#Get Completion
answer = client.gen_completion()
print(answer)

Quantum computing is a revolutionary computing paradigm that harnesses quantum mechanical phenomena like superposition and entanglement to process information in ways that can potentially solve certain complex problems exponentially faster than classical computers.


The following example demonstrates how the context helps. <br>
The user prompt is "Write another sentence" and this prompt without a context is very ambiguous. <br>

In [17]:
client.add_assistant_message(answer)
user_prompt = "Write another sentence"
client.add_user_message(user_prompt)
answer = client.gen_completion()
print(answer)

Quantum computers use quantum bits (qubits) that can exist in multiple states simultaneously, unlike classical bits that are limited to just 0 or 1, enabling them to perform many calculations in parallel.


This example demonstrates how to conversation via Claude API <br>
<ol>
<li> What is 2+3?
<li> Add 7 to that
<li> Divide by 2 and if the result is even print "even" otherwise print "odd"
</ol>

In [25]:
client.clear_history()
i=0
while i < 3:
    user_input=input("?>")
    print("user_input: "+user_input)
    client.add_user_message(user_input)
    answer = client.gen_completion()
    client.add_assistant_message(answer)
    print("------------")
    print(answer)
    print("------------")
    i += 1

user_input: What is 2+5?
------------
7
------------
user_input: Add 7 to that
------------
7 + 7 = 14
------------
user_input: Divide by 2 and if the result is even print "even" otherwise print "odd"
------------
Let me solve this step by step:

1. 14 ÷ 2 = 7
2. 7 is an odd number

So, the result is "odd"
------------


<h2>How does system prompt influence the generated completion?</h2>
The effect of "system" role in the prompt<br>
Here it is used to assign a role to LLM <br>

In [33]:
# Start a new conversation
client.clear_history()
user_prompt = "How to solve x^2+2x=-1 for x?"
client.add_user_message(user_prompt)

# Without system prompt
answer = client.gen_completion()
print(f"Answer without system prompt : "+answer)

# With system prompt
system = """
You are a patient math tutor.
Do not directly answer a student's questions.
Guide them to a solution step by step.
"""
client.set_system_message(system)
answer2 = client.gen_completion()
print(f"Answer with system prompt : "+answer2)

Answer without system prompt : Let's solve this step by step:

1) First, rearrange the equation to standard quadratic form by moving all terms to one side:
   x^2 + 2x + 1 = 0

2) This is a quadratic equation in the form ax^2 + bx + c = 0
   Where a = 1, b = 2, and c = 1

3) We can solve this using the quadratic formula: x = [-b ± √(b^2 - 4ac)] / (2a)

4) Substitute the values:
   x = [-2 ± √(2^2 - 4(1)(1))] / (2(1))

5) Simplify under the square root:
   x = [-2 ± √(4 - 4)] / 2
   x = [-2 ± √0] / 2
   x = [-2 ± 0] / 2

6) This gives two solutions:
   x = -2/2 = -1

Therefore, the solution is x = -1.

You can verify this by plugging -1 back into the original equation:
(-1)^2 + 2(-1) = 1 - 2 = -1 ✓
Answer with system prompt : Let's solve this step by step:

1. First, let's rearrange the equation to standard quadratic form
   • Move all terms to one side of the equation
   • Add 1 to both sides
   x^2 + 2x + 1 = 0

2. This looks like a perfect square trinomial. Can you recognize the patt

The first test, LLM gives the answer and shows how it solved step by step. <br>
But the second answer is adjusted based on LLM is playing "a patient math tutor" and guides the user via questions. <br>